In [1]:
# made by Gibeom LEE, HI LAB
# 1M to 100k.ipynb 와 달리, folder_path안에 있는 모든 csv파일을 전부 축소 시켜줘서, 다른 폴더에 저장해주는 코드입니다.

In [2]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

In [3]:
# 원본 CSV 파일이 있는 폴더 경로
folder_path = r"C:\Users\user\OneDrive - gachon.ac.kr\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\3. Material selection\250808_대전열, 내구도,  wettability 다시\csv파일"

# 결과를 저장할 새 폴더 경로 생성
parent_dir = os.path.dirname(folder_path)
original_folder_name = os.path.basename(folder_path)
new_folder_name = f"minimized_{original_folder_name}"
output_folder = os.path.join(parent_dir, new_folder_name)
os.makedirs(output_folder, exist_ok=True)

# 모든 CSV 파일 경로 수집
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 파라미터 설정
height = 0.01  # 피크 감지 기준 전압 크기 (노이즈 레벨 고려)
num_samples = 10000 # 최종 줄이고자 하는 행 수

for file_path in csv_files:
    try:
        # CSV 파일 읽기 (헤더 없음)
        df = pd.read_csv(file_path, header=None, names=["Time", "Voltage"], skiprows=2)

        # 피크 검출
        peaks_positive, _ = find_peaks(df["Voltage"], height=height, distance=100)
        peaks_negative, _ = find_peaks(-df["Voltage"], height=height, distance=100)
        peaks = np.sort(np.concatenate((peaks_positive, peaks_negative)))

        # 피크 개수 확인
        num_peaks = len(peaks)
        print(f"{os.path.basename(file_path)} → Peak count: {num_peaks}")

        if num_peaks >= num_samples:
            selected_indices = np.sort(np.random.choice(peaks, num_samples, replace=False))
        else:
            num_non_peak_samples = num_samples - num_peaks
            non_peak_indices = np.setdiff1d(np.arange(len(df)), peaks)
            selected_non_peaks = np.linspace(0, len(non_peak_indices) - 1, num_non_peak_samples, dtype=int)
            selected_non_peaks = non_peak_indices[selected_non_peaks]

            selected_indices = np.sort(np.concatenate((peaks, selected_non_peaks)))

        # 축소된 데이터 생성
        filtered_df = df.iloc[selected_indices]

        # 새 파일 이름 및 저장 경로
        filename = os.path.basename(file_path)
        save_path = os.path.join(output_folder, filename)
        filtered_df.to_csv(save_path, index=False, header=False)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

103H_02.csv → Peak count: 8150
103H_04.csv → Peak count: 8214
103H_06.csv → Peak count: 8236
103H_08.csv → Peak count: 8204
103H_10.csv → Peak count: 8235
103H_12.csv → Peak count: 8242
103H_14.csv → Peak count: 8201
103H_16.csv → Peak count: 8230
103H_18.csv → Peak count: 8227
103H_20.csv → Peak count: 8226
103_01.csv → Peak count: 8462
103_02.csv → Peak count: 8183
103_03.csv → Peak count: 8024
103_04.csv → Peak count: 8117
103_05.csv → Peak count: 8168
103_06.csv → Peak count: 8148
103_08.csv → Peak count: 8211
103_10.csv → Peak count: 8206
103_12.csv → Peak count: 8175
103_14.csv → Peak count: 8181
103_16.csv → Peak count: 8170
103_18.csv → Peak count: 8122
103_20.csv → Peak count: 8112
107_02.csv → Peak count: 8037
107_04.csv → Peak count: 8160
107_06.csv → Peak count: 8221
107_08.csv → Peak count: 8160
107_10.csv → Peak count: 8135
Dode_02.csv → Peak count: 8208
Dode_04.csv → Peak count: 8911
Dode_06.csv → Peak count: 8191
Dode_08.csv → Peak count: 8440
Dode_10.csv → Peak count: 